# Implementacija algoritma $AQC(p)$

Ovde ćemo prikazati kako se implementira prvi algoritam koji su predložili Dōng i Lín. Nasumično ćemo generisati retku matricu veličine $16 \times 16$, a onda i vektor $b$ dužine $16$, nakon čega ćemo prikazati kako se rešava sistem jednačina $A x = b$.

In [1]:
import scipy.sparse as sparse
import numpy as np

rows = 16
cols = 16
density = 0.1  # 10% non-zero elements

R = sparse.random(rows, cols, density=density, format='csr', dtype=np.float64)
A = R.T @ R + sparse.diags(np.ones(cols) * 10, format='csr')
A = A.toarray()

In [2]:
from math import sqrt

b = np.random.rand(cols)
c = sqrt(sum(b*b))
b = 1/c * b

Matrica $A$ najverovatnije nije ermitska pozitivno definitna, pa koristimo recept koji su dali autori da se matrica $A$ najpre pretvori u ermitsku.

In [3]:
X = np.array([[0.0, 1.0], [1.0, 0.0]])
Y = np.array([[0.0, -1.0j], [1.0j, 0.0]])
Z = np.array([[1.0, 0.0], [0.0, -1.0]])

Op, Om = 1/2.0 * (X + 1.0j * Y), 1/2.0 * (X - 1.0j * Y)

In [4]:
A1 = np.kron(Op, A) + np.kron(Om, A.conj().T)
b1 = np.kron(np.array([0, 1]), b)
N = 2*cols

Sada generišemo Hamiltonijane $H_0$ i $H_1$. Koristićemo proceduru za slučaj kada $A$ nije pozitivno definitna.

In [5]:
plus = 1/sqrt(2.0) * np.array([1, 1])
Plus = plus.reshape(-1, 1) @ plus.reshape(1, 2)
minus = 1/sqrt(2.0) * np.array([1, -1])
B1 = b1.reshape(-1, 1) @ b1.reshape(1, N)

Qb = np.eye(2*N) - np.kron(Plus, B1)
H0 = np.kron(Op, np.kron(Z, np.eye(N)) @ Qb) + np.kron(Om, Qb @ np.kron(Z, np.eye(N)))
H1 = np.kron(Op, np.kron(X, A1) @ Qb) + np.kron(Om, Qb @ np.kron(X, A1))

Proverimo da je sve dobro tako što ćemo ubaciti sopstvene vrednosti uz $0$ od $H_0$.

In [6]:
#print(H0 @ np.kron(np.kron(np.array([0, 1]), plus), b1).reshape(-1, 1))
#print(H0 @ np.kron(np.kron(np.array([1, 0]), minus), b1).reshape(-1, 1))

Sve je tu negde na oko $10^{-16}$. To je verovatno zbog preciznosti računanja sa **float** promenljivama na računaru. Deluje da je ovo sve u redu.

Sada je potrebno definisati funkciju za protok vremena. Ona zavisi od kondicionog broja matrice $A1$. Pravićemo se da imamo efikasan algoritam za ovo i pustićemo **numpy** da izračuna ovo za nas.

In [7]:
import numpy.linalg as la

kappa = la.cond(A1)
kappa

1.206804173308867

Sada imamo kondicioni broj i možemo definisati funkciju vremena.

In [8]:
def time_function(kappa, s, p):
    return kappa/(kappa - 1) * (1 - (1 + s*(kappa**(p - 1) - 1))**(1/(1 - p)))

Za parametar $p$ ćemo uzeti $0.5$. Sada biramo $T = O(\kappa / \epsilon)$. Neka to bude $T = \kappa / \epsilon$, a za $\epsilon = 1/10$.

In [9]:
epsilon = 1.0/10.0
T = kappa / epsilon

T

12.06804173308867

Ostaje samo da se izabere parametar $M$ koji određuje koliko ćemo imati operatora vremenske evolucije. On se bira tako da bude $M = O(\text{polylog}(N) T^2 / \epsilon)$. Postavićemo ga na $20$, jer nemamo dovoljno jaku mašinu da simulira kvantni računar sa toliko mnogo kvantnih kola. Naime, prava vrednost bi ovde bila preko $1000$.

In [10]:
M = 20

Broj kubita je $\log_2$ od veličine matrice $H_0$. U ovom slučaju je to $7$. Sada pravimo kvantno kolo koje će opisati vremensku evoluciju. Na to kvantno kolo najpre treba inicijalizovati $\ket{0} \ket{-} \ket{b_1}$.

In [11]:
from qiskit.circuit import QuantumCircuit
from math import log2
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.circuit.library import StatePreparation
from qiskit.quantum_info import SparsePauliOp

# predstavljamo H0 i H1 u SparsePauliOp

h0, h1 = SparsePauliOp.from_operator(H0), SparsePauliOp.from_operator(H1)
n = int(log2(H0.shape[0]))

time_evolution = QuantumCircuit(n)

initial_state = np.kron(np.array([1, 0]), np.kron(minus, b1))
stateprep = StatePreparation(initial_state)
time_evolution.append(stateprep, range(n))

h = 1.0/M

for i in reversed(range(1, M + 1)):
    si = i*h
    time_evolution.append(PauliEvolutionGate(h1, time = T * time_function(kappa, si, 0.5) * h), range(n))
    time_evolution.append(PauliEvolutionGate(h0, time = T * (1 - time_function(kappa, si, 0.5)) * h), range(n))

time_evolution.measure_all()

Ostaje sad samo da se odredi vektor $\ket{x''}$.

In [12]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.circuit.library import efficient_su2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
 
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator

service = QiskitRuntimeService(channel = "local")
backend = service.least_busy()

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

sampler = Sampler(mode=backend)

qc = pm.run(time_evolution)

result = sampler.run([(qc, [])]).result()

print(result)

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=1024, num_bits=7>)), metadata={'shots': 1024, 'circuit_metadata': {}})], metadata={'version': 2})


In [15]:
# ovde koristimo neki IBM-ov kod da izdvojimo amplitude iz semplera

counts_int = result[0].data.meas.get_int_counts()
counts_bin = result[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

for x in range(2**n):
    if x not in final_distribution_100_int.keys():
        final_distribution_100_int[x] = 0

y = np.array([sqrt(final_distribution_100_int[x]) for x in range(2**n)])

print(y)
print(sqrt(sum(y * y)))

[0.10364452 0.0625     0.09882118 0.09882118 0.06987712 0.07654655
 0.0625     0.04419417 0.08838835 0.09882118 0.11267348 0.08267973
 0.09375    0.06987712 0.10825318 0.08838835 0.0625     0.04419417
 0.08267973 0.06987712 0.07654655 0.07654655 0.10364452 0.04419417
 0.09882118 0.09375    0.10364452 0.08267973 0.11692679 0.09375
 0.07654655 0.11692679 0.08267973 0.07654655 0.09882118 0.0625
 0.0625     0.08838835 0.09375    0.04419417 0.10364452 0.08838835
 0.0625     0.125      0.08838835 0.08267973 0.10364452 0.11267348
 0.08838835 0.10825318 0.08267973 0.09375    0.08267973 0.07654655
 0.11267348 0.07654655 0.09882118 0.09375    0.08267973 0.08267973
 0.08838835 0.         0.09375    0.0625     0.06987712 0.09882118
 0.06987712 0.08267973 0.08267973 0.08838835 0.07654655 0.06987712
 0.10364452 0.0625     0.08267973 0.10825318 0.10364452 0.06987712
 0.06987712 0.09882118 0.10364452 0.09375    0.08838835 0.09882118
 0.07654655 0.09882118 0.0625     0.08267973 0.09882118 0.10364452
 0

Sada je potrebno izvući vektor $x''$ iz ovoga. Ovaj vektor koji smo dobili je u obliku $\ket{0}\ket{+}\ket{x''}$. Kada izračunamo $\ket{0}\ket{+}$, dobijamo $(1/\sqrt{2}, 1/\sqrt{2}, 0, 0)^T$. Onda je vektor koji smo dobili u obliku $((1/\sqrt{2}) x'', (1/\sqrt{2}) x'', 0, 0)^T$. Dakle, čitanjem prvih $N$ koordinata i množenjem sa $\sqrt{2}$ se može dobiti vektor $x''$.

In [16]:
x_guess = np.array([sqrt(2.0) * y[i] for i in range(N)])
x_guess

array([0.14657549, 0.08838835, 0.13975425, 0.13975425, 0.09882118,
       0.10825318, 0.08838835, 0.0625    , 0.125     , 0.13975425,
       0.15934436, 0.11692679, 0.13258252, 0.09882118, 0.15309311,
       0.125     , 0.08838835, 0.0625    , 0.11692679, 0.09882118,
       0.10825318, 0.10825318, 0.14657549, 0.0625    , 0.13975425,
       0.13258252, 0.14657549, 0.11692679, 0.16535946, 0.13258252,
       0.10825318, 0.16535946])

Sada množimo $A_1$ i $x''$ da bismo dobili konstantu $d$.

In [17]:
temp = (A1 @ x_guess.reshape(-1, 1)).reshape(1, N)

In [18]:
d = 0

for i in range(N):
    if temp[0, i] != 0:
        d += b1[i] / temp[0, i]

d /= N

print(d)

print(1/d * b1)
print(temp)

(0.08491988765068588+0j)
[0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 4.51091711+0.j 2.53486988+0.j 0.37837516+0.j 1.4068264 +0.j
 0.01432965+0.j 2.76521851+0.j 3.94998796+0.j 3.10949188+0.j
 1.34563154+0.j 3.31966348+0.j 4.71010207+0.j 3.98869401+0.j
 2.23921897+0.j 0.26729897+0.j 1.66378001+0.j 4.25158528+0.j]
[[0.98438066+0.j 0.66766341+0.j 1.26801532+0.j 0.98821177+0.j
  1.10678597+0.j 1.16101966+0.j 1.71344322+0.j 0.71999262+0.j
  1.49370256+0.j 1.7591548 +0.j 1.53427994+0.j 1.17775904+0.j
  1.82369511+0.j 1.59553827+0.j 1.24507079+0.j 1.77794184+0.j
  1.59227299+0.j 0.93749522+0.j 1.5137897 +0.j 1.39754249+0.j
  1.01035274+0.j 1.16101966+0.j 1.07758679+0.j 0.72671605+0.j
  1.33423416+0.j 1.80758865+0.j 1.68220727+0.j 1.17780349+0.j
  1.46555821+0.j 1.19365731+0.j 1.74147552+0.j 1.359

Konačno, dobijamo: $x = c * d * x''$.

In [19]:
x = c * d * x_guess

print(x)

[0.03062524+0.j 0.01846772+0.j 0.02920002+0.j 0.02920002+0.j
 0.02064753+0.j 0.02261824+0.j 0.01846772+0.j 0.01305865+0.j
 0.0261173 +0.j 0.02920002+0.j 0.03329315+0.j 0.02443049+0.j
 0.02770157+0.j 0.02064753+0.j 0.03198702+0.j 0.0261173 +0.j
 0.01846772+0.j 0.01305865+0.j 0.02443049+0.j 0.02064753+0.j
 0.02261824+0.j 0.02261824+0.j 0.03062524+0.j 0.01305865+0.j
 0.02920002+0.j 0.02770157+0.j 0.03062524+0.j 0.02443049+0.j
 0.03454993+0.j 0.02770157+0.j 0.02261824+0.j 0.03454993+0.j]


Ova aproksimacija je katastrofalna, jer smo izabrali malo $M$.

In [20]:
A1 @ x.reshape(-1, 1)

array([[0.20567488+0.j],
       [0.1395005 +0.j],
       [0.26493704+0.j],
       [0.20647535+0.j],
       [0.23125005+0.j],
       [0.24258155+0.j],
       [0.35800402+0.j],
       [0.15043408+0.j],
       [0.31209176+0.j],
       [0.36755492+0.j],
       [0.32056994+0.j],
       [0.24607904+0.j],
       [0.38103987+0.j],
       [0.33336915+0.j],
       [0.26014305+0.j],
       [0.37148025+0.j],
       [0.33268691+0.j],
       [0.19587872+0.j],
       [0.31628874+0.j],
       [0.29200024+0.j],
       [0.21110145+0.j],
       [0.24258155+0.j],
       [0.22514922+0.j],
       [0.15183886+0.j],
       [0.2787727 +0.j],
       [0.37767461+0.j],
       [0.35147763+0.j],
       [0.24608833+0.j],
       [0.30621133+0.j],
       [0.2494008 +0.j],
       [0.36386104+0.j],
       [0.2840407 +0.j]])